### Import thư viện

In [1]:
import os
import json
import random

In [2]:
import pandas as pd
import translators as ts

<em>Vui lòng không chạy code này !</em>

### Giữ lại một số lượng file phù hợp

Nhóm sử dụng file train.csv có dung lượng > 1.2GB chứa các tin tức từ 2 kênh thông tin online CNN và Daily Mail. Do hạn chế về việc lưu trữ trên github cũng như các đơn vị tính toán (RAM) trên laptop cá nhân, nhóm sử dụng 100.000 dòng đầu tiên của file csv (tương ứng 100000 bài báo tiếng anh) để làm dữ liệu thực hiện project này.

In [3]:
data = pd.read_csv("../../train.csv", nrows=100000)

In [4]:
data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
99995,0cd7ec4013b0a6dbbc9f15ce8a7ff757db5b8ad9,"By . Deni Kirkova . PUBLISHED: . 05:54 EST, 1 ...",Eleven innocent princesses raunch up as Miley ...
99996,0cd84db8ebc91821d07d910e830d8710021a2d6a,A furious Harvard Business School professor ha...,Ben Edelman is an associate professor of busin...
99997,0cd875596eada9164e7d349d654697c342020b65,"By . Daily Mail Reporter . Lewis Dale, 17, is ...","Lewis Dale, 17, weeps in the dock as he is tol..."
99998,0cd9048c654458e89b9f2a219da1b4d12bc6550f,By . Daily Mail Reporter . PUBLISHED: . 11:09 ...,CCTV shows driver chase a man on a scooter and...


Thực hiện tách từng file .txt nhỏ, phục vụ cho việc lưu trữ, đẩy file lên github với id đại diện cho tên file

In [5]:
# Tạo thư mục để lưu file TXT nếu chưa tồn tại
output_folder = '../data_summarize'  # Thay đổi tên thư mục nếu cần
os.makedirs(output_folder, exist_ok=True)

In [6]:
# Lưu từng dòng dữ liệu thành file TXT
for index, row in data.iterrows():
    record = {
        'article': row['article'].replace('\\"', "'").replace('\n', ''),
        'highlights': row['highlights'].replace('\\"', "'").replace('\n', '')
    }

    output_file_path = os.path.join(output_folder, f"{row['id']}.txt")
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(record, output_file, ensure_ascii=False, indent=2)

print("Done!")

Done!


### Thực hiện translate phần summary tạo target cho mô hình translate

Với phần translate, nhóm sử dụng module translators cũng như đã thử nghiệm một số mô hình pretrained, kết quả dịch không đúng nghĩa, sát nghĩa,... dịch thiếu tokens,... nên nhóm thực hiện gán nhãn bằng tay với công cụ hỗ trợ: google dịch, gpt, poe.com, ...

Do nguồn nhân lực và thời gian còn hạn chế, nhóm chỉ gán nhãn 1000 file bất kỳ để sử dụng cho quá tình fine-tuning model translate.

In [8]:
# Tạo thư mục để lưu file TXT nếu chưa tồn tại
summarize_folder = '../data_summarize'
translate_folder = '../data_translate'  # Thay đổi tên thư mục nếu cần
os.makedirs(translate_folder, exist_ok=True)

In [14]:
files = os.listdir(summarize_folder)

In [16]:
random_files = random.sample(files, 1000)
for file_name in random_files:
    with open(f"{summarize_folder}/{file_name}", 'r', encoding='utf-8') as file:
        content = json.load(file)
        translate_summary = ts.translate_text(content["highlights"], # sử dụng google để dịch
                                            from_language='en', to_language='vi', 
                                            translator='google')
        content["vi"] = translate_summary
        with open(f"{translate_folder}/{file_name}", 'w', encoding='utf-8') as output_file:
            json.dump(content, output_file, ensure_ascii=False, indent=2)
print("Done!")

Done!


---